In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import glob

In [2]:
image_filenames = glob.glob('/home/jhr/Documents/data/dc/train/*.jpg')  # 加载路径

In [3]:
image_filenames[1]

'/home/jhr/Documents/data/dc/train/dog.4188.jpg'

In [4]:
len(image_filenames)

25000

In [5]:
image_filenames = np.random.permutation(image_filenames)  # 乱序路径

In [6]:
# 利用split的切片功能，对数据labels进行编码
labels = [int(p.split('/')[-1].split('.')[0] == 'dog')
          for p in image_filenames]
labels[0:10]

[0, 0, 1, 0, 0, 1, 0, 0, 1, 1]

In [7]:
dataset = tf.data.Dataset.from_tensor_slices((image_filenames, labels))
dataset

<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [8]:
#对图片进行处理
def _pre_read(path, label):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)#解码jpeg，将图片转化为tensor类型
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.resize(image, [200, 200])
    image = tf.reshape(image, [200, 200, 1])
    image = tf.image.per_image_standardization(image)#對每個圖像進行線性縮放，image使其均值為 0，方差為 1。
    label = tf.reshape(label, [1])#reshape a tensor
    return image, label

In [9]:
#数据预处理,乱序，重复，分批次
dataset = dataset.map(_pre_read)
dataset = dataset.shuffle(300)#随机打乱数据
dataset = dataset.repeat()#重复数据集
dataset = dataset.batch(32)#数据分批次

# 创建模型

In [10]:
model = keras.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu',
          input_shape=(200, 200, 1)))  # 卷积
model.add(layers.MaxPool2D(2, 2))  # 图像缩小一半
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2, 2))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPool2D(2, 2))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))  # 全连接层
model.add(layers.Dense(1, activation='sigmoid'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 198, 198, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 99, 99, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 97, 97, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 48, 48, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 64)        36928     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 23, 23, 64)        0

In [12]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc']

              )

In [13]:
steps_per_epoch = 25000//32

In [14]:
history = model.fit(dataset, epochs=10, steps_per_epoch=steps_per_epoch)

Epoch 1/10
781/781 [==============================] - 1118s 1s/step - loss: 0.5430 - acc: 0.7177
Epoch 2/10
  3/781 [..............................] - ETA: 21:05 - loss: 0.4190 - acc: 0.8333

KeyboardInterrupt: 